# Create basic RAG application with Azure AI Search

This sample demonstrates how to create a basic RAG application with Azure AI Search. forked from [azure-search-vector-samples](https://github.com/Azure/azure-search-vector-samples/tree/main)

> ✨ **_Note_** <br>
> Please check the regional support for Azure AI Search before you get started - https://learn.microsoft.com/en-us/azure/search/search-region-support
> In order to use the Semantic Search feature, check your region availability and pricing tier. Make sure it is at least Standard S3.

## Prerequisites

Git clone the repository to your local machine.

```bash
git clone https://github.com/hyogrin/Azure_OpenAI_samples.git
```

-   A subscription key for the Speech service. See [Try the speech service for free](https://docs.microsoft.com/azure/cognitive-services/speech-service/get-started).
-   Python 3.5 or later needs to be installed. Downloads are available [here](https://www.python.org/downloads/).
-   The Python Speech SDK package is available for Windows (x64 or x86) and Linux (x64; Ubuntu 16.04 or Ubuntu 18.04).
-   On Ubuntu 16.04 or 18.04, run the following commands for the installation of required packages:
    ```sh
    sudo apt-get update
    sudo apt-get install libssl3 libasound2
    ```
-   On Debian 9, run the following commands for the installation of required packages:
    ```sh
    sudo apt-get update
    sudo apt-get install libssl1.0.2 libasound2
    ```
-   On Windows you need the [Microsoft Visual C++ Redistributable for Visual Studio 2017](https://support.microsoft.com/help/2977003/the-latest-supported-visual-c-downloads) for your platform.

Configure a Python virtual environment for 3.10 or later:

1.  open the Command Palette (Ctrl+Shift+P).
1.  Search for Python: Create Environment.
1.  select Venv / Conda and choose where to create the new environment.
1.  Select the Python interpreter version. Create with version 3.10 or later.

```bash
python3 -m venv your_env_name
source your_env_name/bin/activate
pip install -r requirements.txt
```

Create an .env file based on the .env-sample file. Copy the new .env file to the folder containing your notebook and update the variables.


In [2]:
import os
import json
import requests

from openai import AzureOpenAI
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.search.documents import SearchIndexingBufferedSender
from azure.search.documents.models import VectorizedQuery
from azure.search.documents.models import VectorizableTextQuery
from azure.search.documents.models import VectorFilterMode
from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    SemanticSearch,
    ComplexField,
    SearchIndex,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters
)

load_dotenv(override=True)

azure_ai_search_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
search_credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_ADMIN_KEY", "")) if len(os.getenv("AZURE_SEARCH_ADMIN_KEY", "")) > 0 else DefaultAzureCredential()

azure_ai_inference_endpoint = os.getenv("AZURE_AI_INFERENCE_ENDPOINT")
azure_ai_inference_key = os.getenv("AZURE_AI_INFERENCE_KEY", "") if len(os.getenv("AZURE_AI_INFERENCE_KEY", "")) > 0 else None
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY", "") if len(os.getenv("AZURE_OPENAI_API_KEY", "")) > 0 else None
azure_openai_deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_embedding_deployment_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME", "text-embedding-ada-002")
azure_openai_embedding_dimensions = int(os.getenv("AZURE_OPENAI_EMBEDDING_DIMENSIONS", 1536))
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-06-01")

index_name = "hotels-sample-index"

'text-embedding-ada-002'

### Create vector index

In [5]:
# vector field - titleVector, contentVector
fields = [
    SimpleField(name="HotelId", type=SearchFieldDataType.String, key=True),
    SearchableField(name="HotelName", type=SearchFieldDataType.String, sortable=True),
    SearchableField(name="Description", type=SearchFieldDataType.String, analyzer_name="en.lucene"),
    # https://learn.microsoft.com/en-us/azure/search/search-language-support
    SearchableField(name="Description_kr", type=SearchFieldDataType.String, analyzer_name="ko.microsoft"),
    SearchableField(name="Description_fr", type=SearchFieldDataType.String, analyzer_name="fr.lucene"),
    SearchableField(name="Category", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),

    SearchableField(name="Tags", collection=True, type=SearchFieldDataType.String, facetable=True, filterable=True),

    SimpleField(name="ParkingIncluded", type=SearchFieldDataType.Boolean, facetable=True, filterable=True, sortable=True),
    SimpleField(name="LastRenovationDate", type=SearchFieldDataType.DateTimeOffset, facetable=True, filterable=True, sortable=True),
    SimpleField(name="Rating", type=SearchFieldDataType.Double, facetable=True, filterable=True, sortable=True),

    ComplexField(name="Address", fields=[
        SearchableField(name="StreetAddress", type=SearchFieldDataType.String),
        SearchableField(name="City", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
        SearchableField(name="StateProvince", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
        SearchableField(name="PostalCode", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
        SearchableField(name="Country", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
    ]),
    SimpleField(name="Location", type=SearchFieldDataType.GeographyPoint, filterable=True, sortable=True),
    ComplexField(name="Rooms", collection=True, fields=[
        SearchableField(name="Description", type=SearchFieldDataType.String, analyzer_name="en.lucene"),
        SearchableField(name="Description_kr", type=SearchFieldDataType.String, analyzer_name="ko.microsoft"),
        SearchableField(name="Description_fr", type=SearchFieldDataType.String, analyzer_name="fr.lucene"),
        SearchableField(name="Type", type=SearchFieldDataType.String, facetable=True, filterable=True),
        SimpleField(name="BaseRate", type=SearchFieldDataType.Double, facetable=True, filterable=True),
        SearchableField(name="BedOptions", type=SearchFieldDataType.String, facetable=True, filterable=True),
        SimpleField(name="SleepsCount", type=SearchFieldDataType.Int32, facetable=True, filterable=True),
        SimpleField(name="SmokingAllowed", type=SearchFieldDataType.Boolean, facetable=True, filterable=True),
        SearchableField(name="Tags", type=SearchFieldDataType.String, collection=True, facetable=True, filterable=True),
    ]),
    
    SearchField(name="hotelNameVector",
                type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, 
                vector_search_dimensions=azure_openai_embedding_dimensions, 
                vector_search_profile_name="myHnswProfile"),
    SearchField(name="descriptionVector", 
                type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, 
                vector_search_dimensions=azure_openai_embedding_dimensions, 
                vector_search_profile_name="myHnswProfile"),
    SearchField(name="descriptionKOVector", 
                type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, 
                vector_search_dimensions=azure_openai_embedding_dimensions, 
                vector_search_profile_name="myHnswProfile"),
]


# Define the vector search configuration
vector_search = VectorSearch(
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
            vectorizer_name="myVectorizer"
        )
    ],
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw"
        )
    ],
    vectorizers=[
        AzureOpenAIVectorizer(
            vectorizer_name = "myVectorizer",
            kind="azureOpenAI",  
            parameters = AzureOpenAIVectorizerParameters(
                resource_url = azure_openai_endpoint,
                deployment_name = azure_openai_embedding_deployment_name,
                model_name = azure_openai_embedding_deployment_name,
                api_key = azure_openai_key
            )
        )
    ]
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="HotelName"),
        keywords_fields=[SemanticField(field_name="Category")],
        content_fields=[SemanticField(field_name="Description")]
    )
)

# Create the semantic search with the configuration  
semantic_search = SemanticSearch(configurations=[semantic_config])

index_client = SearchIndexClient(endpoint=azure_ai_search_endpoint, credential=search_credential)

# Create the search index
index = SearchIndex(
    name = index_name,
    fields = fields,
    vector_search = vector_search, 
    semantic_search = semantic_search)
result = index_client.create_or_update_index(index)

print(f'{result.name} created')

hotels-sample-index created


### Create embedding

Reads the document to index, embeds certain fields (HotelName, Description), and indexes them.

In [6]:
# text-embedding-ada-002 embedding
openai_client = AzureOpenAI(
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key
)

In [11]:
# read the hotels_data.json file and save it to the output directory after embedding.
# You don't need to run this code if you have already run it once.

INDEX_JSON_DATA = True

if(INDEX_JSON_DATA):
    hotels_data_file_path = '../sample-docs/hotels_data_ko.json'

    with open(file=hotels_data_file_path, mode='r', encoding='utf-8-sig') as file:
        hotel_documents = json.load(file)['value']

    # document embedding

    # HotelName, Description embedding
    hotel_name = [item['HotelName'] for item in hotel_documents]
    description = [item['Description'] for item in hotel_documents]
    description_kr = [item['Description_kr'] for item in hotel_documents]

    hotel_name_response = openai_client.embeddings.create(
        model = azure_openai_embedding_deployment_name,
        input = hotel_name
    )
    hotel_name_embeddings = [item.embedding for item in hotel_name_response.data]

    description_response = openai_client.embeddings.create(
        model = azure_openai_embedding_deployment_name, 
        input = description
    )
    description_embeddings = [item.embedding for item in description_response.data]

    description_kr_response = openai_client.embeddings.create(
        model = azure_openai_embedding_deployment_name, 
        input = description_kr
    )
    description_kr_embeddings = [item.embedding for item in description_kr_response.data]

    for i, item in enumerate(hotel_documents):
        item['hotelNameVector'] = hotel_name_embeddings[i]
        item['descriptionVector'] = description_embeddings[i]
        item['descriptionKOVector'] = description_kr_embeddings[i]

    # save the result in docVectors.json 
    output_path = os.path.join('.', 'output', 'docVectors.json')
    output_directory = os.path.dirname(output_path)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    with open(output_path, "w") as f:
        json.dump(hotel_documents, f)

In [12]:
### input vector index
output_path = os.path.join('.', 'output', 'docVectors.json')
output_directory = os.path.dirname(output_path)

if not os.path.exists(output_directory):
    os.makedirs(output_directory)
with open(output_path, 'r') as file:  
    documents = json.load(file)  

search_client = SearchClient(endpoint=azure_ai_search_endpoint, index_name=index_name, credential=search_credential)
result = search_client.upload_documents(documents)
print(f"document indexing: {len(documents)}") 

document indexing: 50


### VectorizableQuery, VectorizableTextQuery
- [VectorizedQuery](https://learn.microsoft.com/ko-kr/python/api/azure-search-documents/azure.search.documents.models.vectorizedquery?view=azure-python)

In [13]:
import time

# vector Search
# query = "WiFi가 제공되는 전통적인 호텔"  
query = "뉴욕에 중심부의의 WiFi가 제공되는 역사있고 전통적인 호텔"  

start_time = time.time()

embedding = openai_client.embeddings.create(
    input = query,
    model = azure_openai_embedding_deployment_name
).data[0].embedding

# Search with embdding query on descriptionVector field to return top 3 closest items
vector_query = VectorizedQuery(
    vector = embedding, 
    k_nearest_neighbors=3, 
    fields = "descriptionVector")
    #fields = "descriptionVector")
  
results = search_client.search(  
    search_text = None,  
    vector_queries = [vector_query],
    select=["HotelName", "Description_kr", "Category"],
)  

print("========== Result with VectorizedQuery ==========")  
for result in results:  
    print(f"HotelName: {result['HotelName']}")  
    print(f"Score: {result['@search.score']}")  
    #print(f"Description: {result['Description']}")  
    print(f"Description_kr: {result['Description_kr']}")  
    print(f"Category: {result['Category']}\n")  

end_time = time.time()
print(f"Time taken for VectorizedQuery search: {end_time - start_time} seconds")

start_time = time.time()

vector_query = VectorizableTextQuery(
    text=query, 
    k_nearest_neighbors=3, 
    fields="descriptionVector"
)
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["HotelName", "Description_kr", "Category"],)  
  
print("========== Result with VectorizableTextQuery  ==========")    
for result in results:
    print(f"HotelName: {result['HotelName']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Description: {result['Description_kr']}")  
    print(f"Category: {result['Category']}\n")      

end_time = time.time()
print(f"Time taken for VectorizableTextQuery search: {end_time - start_time} seconds")    

========== Result with VectorizedQuery ==========
HotelName: Stay-Kay City Hotel
Score: 0.8408806
Description_kr: 이 고전적인 호텔은 완전히 리모델링되었으며 뉴욕의 중심부에 있는 주요 상업 거리의 이상적인 위치에 있습니다. 몇 분 거리에 타임스 스퀘어와 도시의 역사적인 중심지, 그리고 뉴욕을 미국에서 가장 매력적이고 세계적인 도시 중 하나로 만드는 다른 명소들이 있습니다.
Category: Boutique

HotelName: By the Market Hotel
Score: 0.8372628
Description_kr: 지금 예약하고 최대 30% 할인 받으세요. 중심지에 위치해 있으며, 엠파이어 스테이트 빌딩과 타임스 스퀘어에서 도보 거리입니다. 완전히 새로워진 객실. 흠잡을 데 없는 서비스.
Category: Budget

HotelName: Countryside Hotel
Score: 0.8366806
Description_kr: 전통 호텔에서 최대 50% 할인 혜택을 누리세요. 무료 WiFi, 도심 근처의 훌륭한 위치, 완비된 주방, 세탁기 및 건조기, 24시간 지원, 볼링장, 피트니스 센터 등 다양한 편의 시설이 마련되어 있습니다.
Category: Extended-Stay

Time taken for VectorizedQuery search: 0.4198637008666992 seconds
========== Result with VectorizableTextQuery  ==========
HotelName: Stay-Kay City Hotel
Score: 0.8408806
Description: 이 고전적인 호텔은 완전히 리모델링되었으며 뉴욕의 중심부에 있는 주요 상업 거리의 이상적인 위치에 있습니다. 몇 분 거리에 타임스 스퀘어와 도시의 역사적인 중심지, 그리고 뉴욕을 미국에서 가장 매력적이고 세계적인 도시 중 하나로 만드는 다른 명소들이 있습니다.
Categ

### Multi-Vector Search
- Cross-field vector search, which allows you to pass multiple query vectors to query multiple vector fields simultaneously.
- In this case, you can pass query vectors from two different embedding models to the corresponding vector fields in the index.
- For each vector field, you can also give different search settings, such as performing a vector search, weights, exhaustive KNN, etc.

In [14]:
query = "traditional hotels with free wifi"  
  
vector_query_1 = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="hotelNameVector", weight=1, exhaustive=True)
vector_query_2 = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="descriptionVector", weight=0.7)

results = search_client.search(  
    search_text=None,  
    vector_queries=[vector_query_1, vector_query_2],
    select=["HotelName", "Description", "Category"],
)  
  
for result in results:  
    print(f"HotelName: {result['HotelName']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Description: {result['Description']}")  
    print(f"Category: {result['Category']}\n")   

HotelName: Swirling Currents Hotel
Score: 0.01666666753590107
Description: Spacious rooms, glamorous suites and residences, rooftop pool, walking access to shopping, dining, entertainment and the city center. Each room comes equipped with a microwave, a coffee maker and a minifridge. In-room entertainment includes complimentary W-Fi and flat-screen TVs. 
Category: Suite

HotelName: Twin Vortex Hotel
Score: 0.016393441706895828
Description: New experience in the making. Be the first to experience the luxury of the Twin Vortex. Reserve one of our newly-renovated guest rooms today.
Category: Luxury

HotelName: Old Century Hotel
Score: 0.016129031777381897
Description: The hotel is situated in a nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts. The hotel also regularly hosts events like wine tastings, be

### Using filters in Vector search

- Shows how to apply filters to your search.
- You can choose whether to use pre-filtering (the default) or post-filtering.

In [15]:
query = "traditional hotels with free wifi"  
  
vector_query = VectorizableTextQuery(
    text=query,
    k_nearest_neighbors=3,
    fields="descriptionVector")

results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    vector_filter_mode=VectorFilterMode.PRE_FILTER,
    filter="Category eq 'Budget'",
    select=["HotelName", "Description", "Category"],
)
  
for result in results:  
    print(f"HotelName: {result['HotelName']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Description: {result['Description']}")  
    print(f"Category: {result['Category']}\n") 

HotelName: Friendly Motor Inn
Score: 0.8618179
Description: Close to historic sites, local attractions, and urban parks. Free Shuttle to the airport and casinos. Free breakfast and WiFi.
Category: Budget

HotelName: Lion's Den Inn
Score: 0.8600037
Description: Full breakfast buffet for 2 for only $1. Excited to show off our room upgrades, faster high speed WiFi, updated corridors & meeting space. Come relax and enjoy your stay.
Category: Budget

HotelName: Treehouse Hotel
Score: 0.85627013
Description: Near the beating heart of our vibrant downtown and bustling business district. Experience the warmth of our hotel. Enjoy free WiFi, local transportation and Milk & Cookies.
Category: Budget



### Hybrid search
- Performs a combination of Lexical and Vector searches and returns results.
- In the case of Vector search, you can improve the quality of your search by performing a lexical exact search along with a search using similarity.

In [16]:
query = "near downtown hotels"  
  
vector_query = VectorizableTextQuery(
    text=query, 
    k_nearest_neighbors=3, 
    fields="descriptionVector")

# 어휘 검색(search_text=query)과 벡터 검색(vector_queries=[vector_query])을 함께 사용하여 검색 합니다.
results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["HotelName", "Description", "Category"],
    top=3
)  

for result in results:  
    print(f"HotelName: {result['HotelName']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Description: {result['Description']}")  
    print(f"Category: {result['Category']}\n")  

HotelName: Treehouse Hotel
Score: 0.03177805617451668
Description: Near the beating heart of our vibrant downtown and bustling business district. Experience the warmth of our hotel. Enjoy free WiFi, local transportation and Milk & Cookies.
Category: Budget

HotelName: Luxury Lion Resort
Score: 0.03159204125404358
Description: Unmatched Luxury. Visit our downtown hotel to indulge in luxury accommodations. Moments from the stadium and transportation hubs, we feature the best in convenience and comfort.
Category: Luxury

HotelName: Hotel on the Harbor
Score: 0.03021353855729103
Description: Stunning Downtown Hotel with indoor Pool. Ideally located close to theatres, museums and the convention center. Indoor Pool and Sauna and fitness centre. Popular Bar & Restaurant
Category: Luxury



In [17]:
# 위의 검색 결과와, 벡터 검색만 사용(search_text=None) 의 결과를 비교해 보면, 다르다는 것을 알 수 있습니다.
vector_query = VectorizableTextQuery(
    text=query,
    k_nearest_neighbors=3,
    fields="descriptionVector")

results = search_client.search(  
    search_text=None,  
    vector_queries=[vector_query],
    select=["HotelName", "Description", "Category"],
    top=3
)  
  
for result in results:  
    print(f"HotelName: {result['HotelName']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Description: {result['Description']}")  
    print(f"Category: {result['Category']}\n")  

HotelName: Luxury Lion Resort
Score: 0.8876953
Description: Unmatched Luxury. Visit our downtown hotel to indulge in luxury accommodations. Moments from the stadium and transportation hubs, we feature the best in convenience and comfort.
Category: Luxury

HotelName: Treehouse Hotel
Score: 0.88292825
Description: Near the beating heart of our vibrant downtown and bustling business district. Experience the warmth of our hotel. Enjoy free WiFi, local transportation and Milk & Cookies.
Category: Budget

HotelName: Hotel on the Harbor
Score: 0.87755483
Description: Stunning Downtown Hotel with indoor Pool. Ideally located close to theatres, museums and the convention center. Indoor Pool and Sauna and fitness centre. Popular Bar & Restaurant
Category: Luxury



### Semantic Hybrid Search

Hybrid search returns results that are captioned using the semantic ranker.

When creating the vector index, we added a sementic configuration and set the title, content, and keyword fields to use for semantic ranking, captions, highlighting, and answers.

``` python
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="HotelName"),
        keywords_fields=[SemanticField(field_name="Category")],
        content_fields=[SemanticField(field_name="Description")]
    )
)
```



In [18]:
query = "Good hotels for times square"

vector_query = VectorizableTextQuery(
    text=query, 
    k_nearest_neighbors=3, 
    fields="descriptionVector", 
    exhaustive=True)

# Hybrid & Semantic Search
results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["HotelName", "Description", "Category"],
    query_type=QueryType.SEMANTIC, 
    semantic_configuration_name='my-semantic-config', 
    query_caption=QueryCaptionType.EXTRACTIVE, 
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"HotelName: {result['HotelName']}")
    print(f"Reranker Score: {result['@search.reranker_score']}")
    print(f"Description: {result['Description']}")
    print(f"Category: {result['Category']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

Semantic Answer: This<em> classic hotel </em>is<em> fully-refurbished and ideally located on the main commercial artery of the city in the heart of New York.</em> A few minutes away is Times Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities.
Semantic Answer Score: 0.9769999980926514

HotelName: Stay-Kay City Hotel
Reranker Score: 2.979752779006958
Description: This classic hotel is fully-refurbished and ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Times Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities.
Category: Boutique
Caption: This<em> classic hotel </em>is<em> fully-refurbished and ideally located on the main commercial artery of the city in the heart of New York.</em> A few minutes away is Times Square and the h